# Create video index


#### imports 

In [3]:
import polars as pl 
from sentence_transformers import SentenceTransformer

#### load data

In [4]:
df = pl.read_parquet('data/video-transcripts.parquet')
df.head()

video_id,datetime,title,transcript
str,datetime[μs],str,str
"""bZr2vhoXSy8""",2025-02-08 18:10:05,"""I Trained FLUX.1 on My Face (P…","""flux is a state-of-the-art ima…"
"""QvxuR8uLPFs""",2025-02-03 18:00:00,"""How to Build Customer Segments…","""although today's AI models are…"
"""W4s6b2ZM6kI""",2025-01-31 22:38:22,"""Fine-tuning Multimodal Embeddi…","""multimodal embedding models br…"
"""hOLBrIjRAj4""",2025-01-22 21:25:16,"""Fine-Tuning Text Embeddings Fo…","""embedding models represent tex…"
"""V1BR2tb_e8g""",2025-01-13 21:10:47,"""My AI Development Setup (From …","""hey everyone I'm Shaw I just g…"


#### embed titles and transcripts 

In [5]:
model_name = 'all-MiniLM-L6-v2'
column_name_list = ['title','transcript']

In [6]:
model = SentenceTransformer(model_name)

for column_name in column_name_list:
    embedding_arr = model.encode(df[column_name].to_list())

    # store embeddings in a dataframe
    schema_dict = {column_name+'_embedding-'+str(i): float for i in range(embedding_arr.shape[1])}
    df_embedding = pl.DataFrame(embedding_arr, schema=schema_dict)

    # append embeddings to video index
    df = pl.concat([df, df_embedding], how='horizontal')

In [7]:

df.shape

(124, 772)

In [8]:
df.head()

video_id,datetime,title,transcript,title_embedding-0,title_embedding-1,title_embedding-2,title_embedding-3,title_embedding-4,title_embedding-5,title_embedding-6,title_embedding-7,title_embedding-8,title_embedding-9,title_embedding-10,title_embedding-11,title_embedding-12,title_embedding-13,title_embedding-14,title_embedding-15,title_embedding-16,title_embedding-17,title_embedding-18,title_embedding-19,title_embedding-20,title_embedding-21,title_embedding-22,title_embedding-23,title_embedding-24,title_embedding-25,title_embedding-26,title_embedding-27,title_embedding-28,title_embedding-29,title_embedding-30,title_embedding-31,title_embedding-32,…,transcript_embedding-347,transcript_embedding-348,transcript_embedding-349,transcript_embedding-350,transcript_embedding-351,transcript_embedding-352,transcript_embedding-353,transcript_embedding-354,transcript_embedding-355,transcript_embedding-356,transcript_embedding-357,transcript_embedding-358,transcript_embedding-359,transcript_embedding-360,transcript_embedding-361,transcript_embedding-362,transcript_embedding-363,transcript_embedding-364,transcript_embedding-365,transcript_embedding-366,transcript_embedding-367,transcript_embedding-368,transcript_embedding-369,transcript_embedding-370,transcript_embedding-371,transcript_embedding-372,transcript_embedding-373,transcript_embedding-374,transcript_embedding-375,transcript_embedding-376,transcript_embedding-377,transcript_embedding-378,transcript_embedding-379,transcript_embedding-380,transcript_embedding-381,transcript_embedding-382,transcript_embedding-383
str,datetime[μs],str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""bZr2vhoXSy8""",2025-02-08 18:10:05,"""I Trained FLUX.1 on My Face (P…","""flux is a state-of-the-art ima…",-0.10323,-0.063101,0.045255,0.027519,0.049073,-0.130895,0.041504,0.024269,-0.108514,-0.06766,0.002227,-0.084271,0.043815,0.052335,0.019513,-0.01962,0.008622,-0.019429,-0.005975,-0.03834,0.011716,0.070492,0.037155,-0.002652,0.051806,0.001729,0.017232,-0.013841,0.056693,-0.108104,0.012813,0.019935,-0.089308,…,-0.025756,-0.040852,-0.100541,0.027852,-0.032628,0.048355,0.033574,0.004785,-0.024732,-0.117576,-0.070969,-0.060081,-0.010528,-0.01559,0.090648,0.00182,-0.012453,0.089711,0.008851,0.080553,-0.052646,-0.048805,0.04307,-0.025641,-0.047168,0.006739,0.032752,0.045078,-0.018197,0.080647,0.001504,0.104534,0.060422,0.024158,0.07255,-0.011631,-0.074761
"""QvxuR8uLPFs""",2025-02-03 18:00:00,"""How to Build Customer Segments…","""although today's AI models are…",-0.056979,-0.03146,-0.003455,-0.054426,-0.061734,-0.00879,0.04046,0.025732,-0.015264,-0.058018,0.024905,-0.028064,0.052177,-0.000325,0.082099,0.00355,0.044803,-0.011043,-0.03191,-0.029303,-0.020337,-0.000162,-0.049921,-0.008195,-0.097122,0.02876,0.111755,-0.040304,0.020536,0.00057,0.071091,0.076717,0.099094,…,0.032828,-0.053493,-0.005647,-0.01245,0.013892,0.050916,0.023537,0.068807,-0.035857,0.035819,-0.038169,-0.034331,-0.05673,0.003164,0.093093,-0.013914,-0.013554,-0.002032,0.000721,-0.028854,-0.058454,0.016766,0.042245,-0.018605,0.009617,0.061772,0.039387,0.111508,0.016261,0.000445,-0.0035,0.001963,0.126719,0.011522,0.055999,-0.130304,0.009989
"""W4s6b2ZM6kI""",2025-01-31 22:38:22,"""Fine-tuning Multimodal Embeddi…","""multimodal embedding models br…",0.018917,-0.110189,0.087427,-0.031797,0.050235,0.024523,-0.000274,-0.00563,0.021286,-0.064338,-0.0054,-0.005728,0.018725,0.06709,-0.024775,0.012747,0.017184,0.077786,-0.084095,-0.005546,-0.061987,-0.036718,0.096382,-0.127006,0.037827,0.021259,-0.037596,-0.042008,0.071969,-0.028311,0.012695,0.024077,0.00374,…,0.023,-0.005415,-0.038949,-0.008786,0.023704,-0.012237,-0.095138,-0.025574,-0.041069,0.018734,-0.110186,0.003261,-0.016507,0.058545,0.039788,0.010834,-0.03811,0.08

#### save index to file


In [9]:
df.write_parquet('data/video-index.parquet')